# 1. Basic operations

In [4]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../").resolve()))

In [7]:
import pipeGEM as pg
from pipeGEM.data.fetching import list_bigg_model
import cobra

## Get model from BiGG or [Metabolic atlas](https://metabolicatlas.org/)

In [15]:
bigg_models = list_bigg_model()

In [ ]:
https://github.com/SysBioChalmers/Human-GEM/blob/main/model/Human-GEM.mat?raw=true

In [22]:
bigg_models.loc[bigg_models["organism"].str.contains("M* mus"), :]

,bigg_id,gene_count,reaction_count,organism,metabolite_count
77,iMM1415,1375,3726,Mus musculus,2775


In [ ]:
ecoli = cobra.io.load_model()